# Introduction

The project describes a simple ETL data pipeline from a CSV file, and randomly generated ESG data about 8 detergents and their CO2 emissions, and randomly generated sales data of the detergents in different countries.
The CSV file must first be stored as a table in the Databricks data catalog.
(Data source: https://www.kaggle.com/datasets/nelgiriyewithana/countries-of-the-world-2023)

The pipeline generates 2 results in relation to the CO2 footprint caused by detergents:
- CO2 footprint per article per country
- CO2 footprint per country relative to total CO2 footprint per country

Other results could be evaluated, such as CO2 footprint by month and year and per country and per article. But for demonstration purposes, only the 2 above use cases were evaluated.

The detergents data and sales data are related to each other in a star schema structure: detergents table (dimension table) for sales table (facts table). (https://www.databricks.com/blog/2022/05/20/five-simple-steps-for-implementing-a-star-schema-in-databricks-with-delta-lake.html)

Procedural programming was used, because it is mostly structured, reusable and logically encapsulated. Object-oriented would have made less sense, since many individual components work together and should ultimately form a larger whole.

For production usage, the following things should still be changed:
- Put basic core code in a Python repository, and package it, and only call main() in Databricks. This allows changes to be tracked and versioned. Repos can have restricted access (not everyone can simply change production code in the notebook). And code can be used in other notebooks.
- DB credentials not in code, but in secrets manager or key vault

- Data pipeline can generally be triggered (e.g. by Data Factory)
- Serverless functions can also be used

- As a follow-up measure, the DevOps lifecycle could be defined:
  - Development in VS Code, remote access to test cluster in Databricks
  - Repositories in Azure DevOps (or GitHub) for versioning the artifacts
  - Production code linked to Databricks repos
  - Notebooks triggered by Data Factory


In [ ]:
from pyspark.sql import SparkSession  # SparkSession entry point for programming Spark with the Dataset and DataFrame API 
from pyspark import SparkConf  # SparkConf to customize your Spark application configuration before initializing your SparkSession or SparkContext
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
import string

In [ ]:
def create_spark_session():
    conf = SparkConf()   
    conf.setAppName("Databricks Shell")
    conf.setMaster("local[*]")
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
    return spark

spark = create_spark_session()

In [ ]:
def load_world_data(spark):
    path_world_data = "dbfs:/user/hive/warehouse/world_data_2023"
    df_world_data = spark.read.format("delta").load(path_world_data)
    df_world_data_short = df_world_data.select("Country", "Abbreviation", "Co2-Emissions", "Population", "GDP")
    return df_world_data_short

df_world_data_short = load_world_data(spark)
display(df_world_data_short)

Country,Abbreviation,Co2-Emissions,Population,GDP
Afghanistan,AF,"8,672","38,041,754","$19,101,353,833"
Albania,AL,"4,536","2,854,191","$15,278,077,447"
Algeria,DZ,"150,006","43,053,054","$169,988,236,398"
Andorra,AD,469,"77,142","$3,154,057,987"
Angola,AO,"34,693","31,825,295","$94,635,415,870"
Antigua and Barbuda,AG,557,"97,118","$1,727,759,259"
Argentina,AR,"201,348","44,938,712","$449,663,446,954"
Armenia,AM,"5,156","2,957,731","$13,672,802,158"
Australia,AU,"375,908","25,766,605","$1,392,680,589,329"
Austria,AT,"61,448","8,877,067","$446,314,739,528"


In [ ]:
def create_detergents_dataframe():
    n_detergents = 8
    detergents = ["Detergent_" + x for x in string.ascii_uppercase[:n_detergents]]
    prices = np.random.uniform(5, 20, n_detergents)
    emissions = np.random.rand(n_detergents)
    df_detergents = pd.DataFrame({
        "detergent": detergents,
        "price": prices,
        "co2emission": emissions
    })
    return spark.createDataFrame(df_detergents)

df_detergents = create_detergents_dataframe()
display(df_detergents)

detergent,price,co2emission
Detergent_A,17.420960280189746,0.9616346998255387
Detergent_B,12.398255223514354,0.6088777795126956
Detergent_C,15.38816300110926,0.16851325124285288
Detergent_D,12.196656683679127,0.1527578941910609
Detergent_E,18.31587830145941,0.008043397091225923
Detergent_F,18.15053750603291,0.6782694693879415
Detergent_G,10.615604054675238,0.08814038438373839
Detergent_H,19.604752483181542,0.07840557379224378


In [ ]:
def create_sales_dataframe(country_codes, detergents):
    n_sales = 2000
    df_sales = pd.DataFrame({
        "order_number": range(n_sales),
        "article": np.random.choice(detergents, n_sales),
        "quantity": np.random.binomial(30, 0.1, n_sales) + 1,
        "year": np.random.choice(range(2000, 2023), n_sales),
        "month": np.random.choice(range(1, 12), n_sales),
        "country_abbr": np.random.choice(country_codes, n_sales)
    })
    return spark.createDataFrame(df_sales)

# Get unique country codes list
country_codes = [row[0] for row in df_world_data_short.select("Abbreviation").distinct().collect()]

df_sales = create_sales_dataframe(country_codes, df_detergents.select("detergent").rdd.flatMap(lambda x: x).collect())
display(df_sales)


order_number,article,quantity,year,month,country_abbr
0,Detergent_C,3,2010,11,PH
1,Detergent_E,3,2010,2,ZM
2,Detergent_A,2,2008,5,KH
3,Detergent_F,5,2001,8,MC
4,Detergent_H,6,2021,3,KN
5,Detergent_H,3,2008,2,NE
6,Detergent_H,4,2000,9,PG
7,Detergent_A,5,2009,11,CM
8,Detergent_G,5,2013,10,TZ
9,Detergent_F,3,2000,4,GQ


In [ ]:
df_sales_extended = df_sales.join(df_detergents, df_sales.article == df_detergents.detergent, "inner") \
                            .drop("detergent") \
                            .withColumnRenamed("price", "price_per_article") \
                            .withColumnRenamed("co2emission", "co2emission_per_article")

df_grouped_article_country = df_sales_extended.groupBy("article", "country_abbr").agg(
    F.sum(F.col("co2emission_per_article") * F.col("quantity")).alias("Total_co2emission_per_article_per_country"),
    F.sum(F.col("price_per_article") * F.col("quantity")).alias("Total_price_per_article_per_country")
)

display(df_grouped_article_country)


article,country_abbr,Total_co2emission_per_article_per_country,Total_price_per_article_per_country
Detergent_G,PY,1.0576846126048607,127.38724865610286
Detergent_E,MR,0.02413019127367777,54.94763490437823
Detergent_G,HU,0.7932634594536455,95.54043649207713
Detergent_C,BG,1.348106009942823,123.10530400887409
Detergent_A,CL,9.616346998255388,174.20960280189746
Detergent_A,US,11.539616397906464,209.05152336227695
Detergent_E,BO,0.04826038254735554,109.89526980875647
Detergent_E,GD,0.056303779638581464,128.21114811021587
Detergent_C,BH,2.3591855173999403,215.43428201552965
Detergent_H,NE,1.1760836068836567,294.07128724772315


In [ ]:
# company co2 footprint per country relative to total co2 footprint per country
df_grouped_country = df_grouped_article_country.groupBy("country_abbr").agg(F.sum("Total_co2emission_per_article_per_country") \
    .alias("Total_co2emission_per_country")
)

df_grouped_total_compare = df_grouped_country.join(df_world_data_short, on=df_grouped_country.country_abbr == df_world_data_short.Abbreviation, how="inner")\
               .drop("Abbreviation") \
               .withColumnRenamed("Total_co2emission_per_country", "co2emission_by_company") \
               .withColumnRenamed("Co2-Emissions", "Country-Total-Co2-Emissions")
display(df_grouped_total_compare)

country_abbr,co2emission_by_company,Country,Country-Total-Co2-Emissions,Population,GDP
DZ,15.543569515966068,Algeria,"150,006","43,053,054","$169,988,236,398"
MM,15.184096644656714,Myanmar,"25,280","54,045,420","$76,085,852,617"
LT,11.382430301420928,Lithuania,"12,963","2,786,844","$54,219,315,600"
FI,15.763031900586597,Finland,"45,871","5,520,314","$268,761,201,365"
AZ,10.485624798788756,Azerbaijan,"37,620","10,023,318","$39,207,000,000"
UA,19.13396254540514,Ukraine,"202,250","44,385,155","$153,781,069,118"
ZM,11.078515704466941,Zambia,"5,141","17,861,030","$23,064,722,446"
SL,13.544961844870265,Sierra Leone,"1,093","7,813,215","$3,941,474,311"
SB,3.6655306774947083,Solomon Islands,169,"669,823","$1,425,074,226"
LA,15.64291715369331,Laos,"17,763","7,169,455","$18,173,839,128"


In [ ]:
# SQL Server connection properties
server = ""
database = ""
username = ""
password = ""

url = f"jdbc:sqlserver://{server}:1433;database={database}"
properties = {"user": username, "password": password, "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}

In [ ]:
# write to database
df_grouped_article_country.write.jdbc(url=url, table="SalesDataAggregated", mode="append", properties=properties)
df_grouped_total_compare.write.jdbc(url=url, table="Co2EmissionsCompare", mode="append", properties=properties)